# OpenAI 함수 호출을 사용한 간단한 ReAct 에이전트

## ReAct 루프(ReAct Loop)

[ReAct 루프](https://arxiv.org/abs/2210.03629)는 **생각(Thought) -> 행동(Action) -> 관찰(Observation)**의 순환 구조로, 추론과 행동을 번갈아 수행합니다. 이 구조는 LLM 기반 에이전트의 기초로서 매우 성공적이었으며, 에이전트가 여러 단계의 작업을 반복적으로 해결할 수 있게 해줍니다.

In [2]:
import os
from dotenv import load_dotenv  

!echo "OPENAI_API_KEY=<Open AI 발급한 Key를 입력하세요." >> .env #최초만 설정하여 실행
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
%pip install "annotated_docs @ git+https://git@github.com/peterroelants/annotated-docs.git@v0.0.4"

In [4]:
import json
import requests
from collections.abc import Callable
from typing import Annotated as A, Literal as L
from annotated_docs.json_schema import as_json_schema

In [5]:
from openai import OpenAI

MODEL = "gpt-5-mini"
client = OpenAI()

## 에이전트(Agent) 구현


### 함수 정의하기

먼저 에이전트가 호출할 수 있는 몇 가지 간단한 예제 함수들을 정의합니다. 에이전트가 호출할 수 있는 함수는 다음과 같습니다:
- `get_current_location`: 에이전트의 현재 위치를 찾습니다 (IP 주소 기반).
- `get_current_weather`: 주어진 위치의 현재 날씨를 찾습니다.
- `calculate`: 단위 변환과 같은 계산을 돕습니다.
- `finish`: 작업을 마치고 최종 답변을 생성합니다. 이 함수는 `StopException` 예외를 발생시켜 에이전트의 작업이 완료되었음을 쉽게 감지하도록 합니다.

파이썬의 `typing` 라이브러리를 사용하여 함수 인자에 LLM에게 유용한 추가 정보를 어노테이션(annotation)으로 달아줍니다. 특히 `typing.Annotated`를 사용해 인자에 대한 설명을 추가하고, `typing.Literal`을 사용해 인자가 가질 수 있는 특정 값들을 지정합니다.

In [6]:
class StopException(Exception):
    """
    이 예외를 발생시켜 실행을 중지합니다 (작업이 완료되었음을 알리는 신호).
    """
    pass


def finish(answer: A[str, "사용자 질문에 대한 최종 답변입니다."]) -> None:
    """사용자의 질문에 답변하고 대화를 종료합니다."""
    raise StopException(answer)


def get_current_location() -> str:
    """사용자의 현재 위치(위도, 경도)를 가져옵니다."""
    # IP 주소 기반으로 위치 정보를 제공하는 외부 API를 호출합니다.
    return json.dumps(requests.get("http://ip-api.com/json?fields=lat,lon").json())


def get_current_weather(
    latitude: float,
    longitude: float,
    temperature_unit: L["celsius", "fahrenheit"], # 온도는 'celsius' 또는 'fahrenheit'만 가능함을 명시합니다.
) -> str:
    """주어진 위치의 현재 날씨를 가져옵니다."""
    # 날씨 정보를 제공하는 외부 API를 호출합니다.
    resp = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={
            "latitude": latitude,
            "longitude": longitude,
            "temperature_unit": temperature_unit,
            "current_weather": True,
        },
    )
    return json.dumps(resp.json())


def calculate(
    # A[str, "..."]는 LLM에게 이 인자가 어떤 역할을 하는지 설명을 제공합니다.
    formula: A[str, "결과를 계산하기 위한 파이썬 문법의 숫자 표현식입니다."],
) -> str:
    """주어진 수식의 결과를 계산합니다."""
    # 참고: eval() 함수는 임의의 코드를 실행할 수 있어 보안에 취약할 수 있습니다.
    # 실제 애플리케이션에서는 안전한 대안을 사용해야 합니다.
    return str(eval(formula))

### OpenAI에 함수 명세 제공하기

OpenAI 함수 호출 기능을 사용하려면, 우리가 정의한 함수들의 명세를 [JSON 스키마(JSON Schema)](https://json-schema.org/) 형식으로 API에 제공해야 합니다.

JSON 스키마는 데이터의 구조를 정의하는 표준적인 방법입니다. LLM은 이 스키마를 보고 각 함수의 이름, 설명, 필요한 인자, 그리고 각 인자의 데이터 타입이 무엇인지 정확하게 파악할 수 있습니다.

여기서는 `annotated-docs` 라이브러리의 `as_json_schema` 함수를 사용하여 파이썬 함수의 어노테이션을 JSON 스키마로 변환합니다. `as_json_schema`는 `typing` 어노테이션을 활용하여 LLM에게 인자에 대한 추가 정보를 제공합니다.

In [7]:
# LLM 에이전트가 호출할 수 있는 모든 함수입니다.
# 함수의 이름을 문자열 키로, 실제 함수 객체를 값으로 하는 딕셔너리를 만듭니다.
# 이를 통해 LLM이 반환하는 함수 이름으로 실제 파이썬 함수를 쉽게 찾아 실행할 수 있습니다.
name_to_function_map: dict[str, Callable] = {
    get_current_location.__name__: get_current_location,
    get_current_weather.__name__: get_current_weather,
    calculate.__name__: calculate,
    finish.__name__: finish,
}

# 모든 함수에 대한 JSON 스키마를 생성합니다.
# 이 리스트를 OpenAI API에 전달하여 모델이 어떤 함수를 사용할 수 있는지 알려줍니다.
function_schemas = [
    {"function": as_json_schema(func), "type": "function"}
    for func in name_to_function_map.values()
]

# 생성된 JSON 스키마를 출력하여 확인합니다.
for schema in function_schemas:
    print(json.dumps(schema, indent=2))

{
  "function": {
    "name": "get_current_location",
    "description": "\uc0ac\uc6a9\uc790\uc758 \ud604\uc7ac \uc704\uce58(\uc704\ub3c4, \uacbd\ub3c4)\ub97c \uac00\uc838\uc635\ub2c8\ub2e4.",
    "parameters": {
      "properties": {},
      "type": "object"
    }
  },
  "type": "function"
}
{
  "function": {
    "name": "get_current_weather",
    "description": "\uc8fc\uc5b4\uc9c4 \uc704\uce58\uc758 \ud604\uc7ac \ub0a0\uc528\ub97c \uac00\uc838\uc635\ub2c8\ub2e4.",
    "parameters": {
      "properties": {
        "latitude": {
          "type": "number"
        },
        "longitude": {
          "type": "number"
        },
        "temperature_unit": {
          "enum": [
            "celsius",
            "fahrenheit"
          ],
          "type": "string"
        }
      },
      "required": [
        "latitude",
        "longitude",
        "temperature_unit"
      ],
      "type": "object"
    }
  },
  "type": "function"
}
{
  "function": {
    "name": "calculate",
    "descrip

### LLM 에이전트에게 할 질문 정의하기

In [8]:
QUESTION_PROMPT = "\
    현재 위치의 날씨는 어떤가요? 온도는 섭씨로, 풍속은 노트(knot)로 알려주세요."

### ReAct 루프 실행하기

ReAct 루프의 각 단계는 다음 과정으로 구성됩니다:
1.  **모델 호출**: 현재까지의 대화 내용이 담긴 `messages` 리스트를 LLM 에이전트에게 보내 응답을 받습니다.
2.  **응답 추가**: LLM의 응답(주로 함수를 호출하라는 내용)을 `messages` 리스트에 추가합니다.
3.  **함수 실행 및 결과 추가**: 응답에 포함된 각 함수 호출 요청에 따라, 해당 함수를 실제로 실행하고 그 결과를 다시 `messages` 리스트에 추가합니다.
4.  **반복**: 작업이 끝날 때까지 1-3 과정을 반복합니다.

In [ ]:
# 초기 "대화" 메시지 리스트입니다.
messages = [
    {
        "role": "system",
        # 시스템 메시지는 LLM 에이전트의 역할과 행동 방식을 정의하는 중요한 부분입니다.
        "content": "당신은 함수를 순차적으로 호출하여 여러 단계의 질문에 답할 수 있는 유용한 어시스턴트입니다. 생각(THOUGHT, 다음에 호출할 함수를 단계별로 추론), 행동(ACTION, 최종 답변을 위한 다음 단계로 함수 호출), 관찰(OBSERVATION, 함수 결과)의 패턴을 따르세요. 답변에 도달하기 위해 어떤 행동을 취할지 단계별로 추론하세요. 사용자의 발언이나 다른 함수의 출력에서 그대로 가져온 인수로만 함수를 호출하세요.",
    },
    {
        "role": "user",
        "content": QUESTION_PROMPT,
    },
]


def run(messages: list[dict]) -> list[dict]:
    """
    OpenAI 함수 호출을 사용하여 ReAct 루프를 실행합니다.
    """
    # 무한 루프를 방지하기 위해 최대 반복 횟수를 설정합니다.
    max_iterations = 20
    for i in range(max_iterations):
        # 1. 메시지 리스트를 보내 다음 응답을 받습니다.
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=function_schemas, # 사용 가능한 함수 목록(JSON 스키마)을 전달합니다.
            tool_choice="auto", # 모델이 함수를 호출할지 스스로 결정하도록 합니다.
        )
        response_message = response.choices[0].message
        # 2. 어시스턴트의 응답을 대화 기록에 추가합니다.
        messages.append(response_message)
        
        # 3. 모델이 함수 호출을 원했는지 확인합니다.
        tool_calls = response_message.tool_calls
        if tool_calls:
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                
                # 함수 이름이 유효한지 확인합니다.
                if function_name not in name_to_function_map:
                    print(f"잘못된 함수 이름입니다: {function_name}")
                    messages.append(
                        {
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "name": function_name,
                            "content": f"잘못된 함수 이름입니다: {function_name!r}",
                        }
                    )
                    continue
                
                # 호출할 함수를 가져옵니다.
                function_to_call: Callable = name_to_function_map[function_name]
                
                # 함수 인자를 파싱합니다.
                try:
                    function_args_dict = json.loads(tool_call.function.arguments)
                except json.JSONDecodeError as exc:
                    print(f"함수 인자 디코딩 오류: {exc}")
                    messages.append(
                        {
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "name": function_name,
                            "content": f"함수 `{function_name}`의 인자 {tool_call.function.arguments!r} 디코딩 오류! 오류: {exc!s}",
                        }
                    )
                    continue
                
                # 생성된 인자로 선택된 함수를 호출합니다.
                try:
                    print(
                        f"{function_name} 함수 호출 중, 인자: {json.dumps(function_args_dict)}"
                    )
                    function_response = function_to_call(**function_args_dict)
                    
                    # 3.1. 함수 응답(관찰 결과)을 대화 기록에 추가합니다.
                    messages.append(
                        {
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "name": function_name,
                            "content": function_response,
                        }
                    )
                except StopException as exc:
                    # 에이전트가 대화를 멈추고 싶어하는 경우 (예상된 동작)
                    print(f"작업 완료, 메시지: '{exc!s}'")
                    return messages
                except Exception as exc:
                    # 예상치 못한 함수 호출 오류
                    print(
                        f"함수 `{function_name}` 호출 오류: {type(exc).__name__}: {exc!s}"
                    )
                    messages.append(
                        {
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "name": function_name,
                            "content": f"함수 `{function_name}` 호출 오류: {type(exc).__name__}: {exc!s}!",
                        }
                    )
                    continue
    return messages


messages = run(messages)

get_current_location 함수 호출 중, 인자: {}
get_current_weather 함수 호출 중, 인자: {"latitude": 37.3654, "longitude": 127.122, "temperature_unit": "celsius"}
calculate 함수 호출 중, 인자: {"formula": "4.2/1.852"}
finish 함수 호출 중, 인자: {"answer": "\ud604\uc7ac \uc704\uce58(\uc704\ub3c4 37.3654, \uacbd\ub3c4 127.122) \uadfc\ucc98\uc758 \ud604\uc7ac \ub0a0\uc528\uc785\ub2c8\ub2e4:\n- \uae30\uc628: 25.5 \u00b0C\n- \ud48d\uc18d: \uc57d 2.27 \ub178\ud2b8 (\uc6d0\ub798 4.2 km/h)\n- \ud48d\ud5a5: 200\u00b0 (\ub300\uccb4\ub85c \ub0a8\ub0a8\uc11c)\n- \ub0a0\uc528 \uc0c1\ud0dc: \ubd80\ubd84\uc801\uc73c\ub85c \ud750\ub9bc (\uae30\uc0c1 \ucf54\ub4dc 2)\n\n\uc6d0\ud558\uc2dc\uba74 \ud48d\uc18d\u00b7\uc628\ub3c4\uc5d0 \ub300\ud55c \ub2e4\ub978 \ub2e8\uc704 \ubcc0\ud658\uc774\ub098 \ud5a5\ud6c4 \uc608\ubcf4\ub3c4 \uc54c\ub824\ub4dc\ub9b4\uac8c\uc694."}
작업 완료, 메시지: '현재 위치(위도 37.3654, 경도 127.122) 근처의 현재 날씨입니다:
- 기온: 25.5 °C
- 풍속: 약 2.27 노트 (원래 4.2 km/h)
- 풍향: 200° (대체로 남남서)
- 날씨 상태: 부분적으로 흐림 (기상 코드 2)

원하시면 풍속·온도에 대한 다른 단위 변

In [17]:
# 최종 messages 리스트의 모든 내용을 출력합니다.
# 이 기록을 통해 에이전트가 어떤 생각과 행동, 관찰을 거쳐 결론에 도달했는지 전체 흐름을 파악할 수 있습니다.
for message in messages:
    if not isinstance(message, dict):
        message = message.model_dump()  # Pydantic 모델을 dict로 변환합니다.
    print(json.dumps(message, indent=2))

{
  "role": "system",
  "content": "\ub2f9\uc2e0\uc740 \ud568\uc218\ub97c \uc21c\ucc28\uc801\uc73c\ub85c \ud638\ucd9c\ud558\uc5ec \uc5ec\ub7ec \ub2e8\uacc4\uc758 \uc9c8\ubb38\uc5d0 \ub2f5\ud560 \uc218 \uc788\ub294 \uc720\uc6a9\ud55c \uc5b4\uc2dc\uc2a4\ud134\ud2b8\uc785\ub2c8\ub2e4. \uc0dd\uac01(THOUGHT, \ub2e4\uc74c\uc5d0 \ud638\ucd9c\ud560 \ud568\uc218\ub97c \ub2e8\uacc4\ubcc4\ub85c \ucd94\ub860), \ud589\ub3d9(ACTION, \ucd5c\uc885 \ub2f5\ubcc0\uc744 \uc704\ud55c \ub2e4\uc74c \ub2e8\uacc4\ub85c \ud568\uc218 \ud638\ucd9c), \uad00\ucc30(OBSERVATION, \ud568\uc218 \uacb0\uacfc)\uc758 \ud328\ud134\uc744 \ub530\ub974\uc138\uc694. \ub2f5\ubcc0\uc5d0 \ub3c4\ub2ec\ud558\uae30 \uc704\ud574 \uc5b4\ub5a4 \ud589\ub3d9\uc744 \ucde8\ud560\uc9c0 \ub2e8\uacc4\ubcc4\ub85c \ucd94\ub860\ud558\uc138\uc694. \uc0ac\uc6a9\uc790\uc758 \ubc1c\uc5b8\uc774\ub098 \ub2e4\ub978 \ud568\uc218\uc758 \ucd9c\ub825\uc5d0\uc11c \uadf8\ub300\ub85c \uac00\uc838\uc628 \uc778\uc218\ub85c\ub9cc \ud568\uc218\ub97c \ud638\ucd9c\u